conda create -n tft_env python=3.7

In [1]:
import argparse
import datetime as dte
import os
import numpy as np 
import pandas as pd
import tensorflow.compat.v1 as tf

In [3]:
import sklearn.preprocessing
import keras

AttributeError: module 'tensorflow_core.compat.v2' has no attribute '__internal__'

In [2]:
#import data_formatters.base
#import expt_settings.configs
#import libs.hyperparam_opt
import libs.tft_model
import libs.utils as utils

data_formatter_class = {
        'volatility': data_formatters.volatility.VolatilityFormatter,
        'electricity': data_formatters.electricity.ElectricityFormatter,
        'traffic': data_formatters.traffic.TrafficFormatter,
        'favorita': data_formatters.favorita.FavoritaFormatter
    }
formatter = data_formatter_class('favorita')

In [3]:
import tensorflow.python.keras.backend as K
default_keras_session = K.get_session()
# cpu
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # for training on cpu
tf_config = tf.ConfigProto(
        log_device_placement=False, device_count={'GPU': 0})

if use_gpu:
    tf_config = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

else:
    tf_config = utils.get_default_tensorflow_config(tf_device="cpu")


NameError: name 'use_gpu' is not defined

In [4]:
data_csv_path = 'favorita_out.csv'
df = pd.read_csv(data_csv_path, index_col=0)

### Params

In [5]:
params = {
        'dropout_rate': 0.1,
        'hidden_layer_size': 240,
        'learning_rate': 0.001,
        'minibatch_size': 128,
        'max_gradient_norm': 100.,
        'num_heads': 4,
        'stack_size': 1
    }

fixed_params = {
        'total_time_steps': 120,
        'num_encoder_steps': 90,
        'num_epochs': 100,
        'early_stopping_patience': 5,
        'multiprocessing_workers': 5
    }

In [6]:
import abc
import enum
class DataTypes(enum.IntEnum):
  """Defines numerical types of each column."""
  REAL_VALUED = 0
  CATEGORICAL = 1
  DATE = 2

class InputTypes(enum.IntEnum):
  """Defines input types of each column."""
  TARGET = 0
  OBSERVED_INPUT = 1
  KNOWN_INPUT = 2
  STATIC_INPUT = 3
  ID = 4  # Single column used as an entity identifier
  TIME = 5 

column_definition = [
      ('traj_id', DataTypes.REAL_VALUED, InputTypes.ID),
      ('date', DataTypes.DATE, InputTypes.TIME),

      ('log_sales', DataTypes.REAL_VALUED, InputTypes.TARGET),

      ('onpromotion', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
      ('day_of_week', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
      ('national_hol', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
      ('regional_hol', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
      ('local_hol', DataTypes.CATEGORICAL, InputTypes.KNOWN_INPUT),
      
      ('transactions', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),
      ('oil', DataTypes.REAL_VALUED, InputTypes.OBSERVED_INPUT),

      ('day_of_month', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
      ('month', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),
      ('open', DataTypes.REAL_VALUED, InputTypes.KNOWN_INPUT),

      ('item_nbr', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
      ('store_nbr', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
      ('city', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
      ('state', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
      ('type', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
      ('cluster', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
      ('family', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
      ('class', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT),
      ('perishable', DataTypes.CATEGORICAL, InputTypes.STATIC_INPUT)
  ]

In [7]:
def _check_single_column(input_type):
      length = len([tup for tup in column_definition if tup[2] == input_type])

      if length != 1:
        raise ValueError('Illegal number of inputs ({}) of type {}'.format(
            length, input_type))

_check_single_column(InputTypes.ID)
_check_single_column(InputTypes.TIME)

In [8]:
identifier = [tup for tup in column_definition if tup[2] == InputTypes.ID]
time = [tup for tup in column_definition if tup[2] == InputTypes.TIME]
real_inputs = [
        tup for tup in column_definition if tup[1] == DataTypes.REAL_VALUED and
        tup[2] not in {InputTypes.ID, InputTypes.TIME}
    ]

In [9]:
col_definition_map = {tup[0]: tup for tup in column_definition}
col_order = [
        'item_nbr', 'store_nbr', 'city', 'state', 'type', 'cluster', 'family',
        'class', 'perishable', 'onpromotion', 'day_of_week', 'national_hol',
        'regional_hol', 'local_hol'
    ]
categorical_inputs = [
        col_definition_map[k] for k in col_order if k in col_definition_map
    ]

In [10]:
column_definitions = identifier + time + real_inputs + categorical_inputs
fixed_params['column_definition'] = identifier + time + real_inputs + categorical_inputs

### Split data

In [11]:
df.date.sort_values()

2206    2013-01-01
817     2013-01-02
2207    2013-01-02
818     2013-01-03
2208    2013-01-03
           ...    
1634    2015-03-30
815     2015-03-30
3095    2015-03-31
3025    2015-03-31
816     2015-03-31
Name: date, Length: 3096, dtype: object

In [21]:
valid_boundary = dte.datetime(2013, 12, 1)

In [22]:
time_steps = fixed_params['total_time_steps'] # 120
lookback = fixed_params['num_encoder_steps'] # 90
forecast_horizon = time_steps - lookback # forecast_horizon=30

In [28]:
df.head()

,store_nbr,item_nbr,unit_sales,onpromotion,traj_id,unique_id,open,date,log_sales,oil,...,family,class,perishable,transactions,day_of_week,day_of_month,month,national_hol,regional_hol,local_hol
0,1.0,103520.0,2.0,NaN,1_103520,1_103520_2013-01-04 00:00:00,1.0,2013-01-04,0.693147,93.12,...,GROCERY I,1028,0,1863.0,4,4,1,NaN,NaN,NaN
1,1.0,103520.0,3.0,NaN,1_103520,1_103520_2013-01-05 00:00:00,1.0,2013-01-05,1.098612,-1.00,...,GROCERY I,1028,0,1509.0,5,5,1,Recupero puente Navidad,NaN,NaN
3,1.0,103520.0,2.0,NaN,1_103520,1_103520_2013-01-07 00:00:00,1.0,2013-01-07,0.693147,93.20,...,GROCERY I,1028,0,1807.0,0,7,1,NaN,NaN,NaN
4,1.0,103520.0,6.0,NaN,1_103520,1_103520_2013-01-08 00:00:00,1.0,2013-01-08,1.791759,93.21,...,GROCERY I,1028,0,1869.0,1,8,1,NaN,NaN,NaN
5,1.0,103520.0,3.0,NaN,1_103520,1_103520_2013-01-09 00:00:00,1.0,2013-01-09,1.098612,93.08,...,GROCERY I,1028,0,1910.0,2,9,1,NaN,NaN,NaN


In [25]:
df['date'] = pd.to_datetime(df['date'])
df_lists = {'train': [], 'valid': [], 'test': []}
for _, sliced in df.groupby('traj_id'):
      index = sliced['date']
      train = sliced.loc[index < valid_boundary]
      train_len = len(train)
      valid_len = train_len + forecast_horizon # valid_len = train_len + 30
      valid = sliced.iloc[train_len - lookback:valid_len, :]
      test = sliced.iloc[valid_len - lookback:valid_len + forecast_horizon, :]

      sliced_map = {'train': train, 'valid': valid, 'test': test}

      for k in sliced_map:
        item = sliced_map[k]

        if len(item) >= time_steps:
          df_lists[k].append(item)

# time_steps = fixed_params['total_time_steps'] # 120
# lookback = fixed_params['num_encoder_steps'] # 90
# forecast_horizon = time_steps - lookback # forecast_horizon=30

In [31]:
dfs = {k: pd.concat(df_lists[k], axis=0) for k in df_lists} # k= train,valid,test

In [36]:
train = dfs['train']

In [37]:
l = [tup[0] for tup in column_definition if tup[2] == InputTypes.ID]
id_column = l[0]  # l[0]=='traj_id'
identifiers = list(train[id_column].unique())
identifiers

['1_103520', '1_103665', '1_96995', '25_103665']

In [38]:
def filter_ids(frame): # must have same traj_id among train, valid and test dataset
    index = frame['traj_id']
    return frame.loc[index.apply(lambda x: x in set(identifiers))] 

In [39]:
valid = filter_ids(dfs['valid'])
test = filter_ids(dfs['test'])
print(train.shape)
print(valid.shape)
print(test.shape)

(921, 24)
(360, 24)
(360, 24)


### Transform inputs

#### Column name

In [40]:
column_definitions = get_column_definition()

NameError: name 'get_column_definition' is not defined

In [38]:
id_column = utils.get_single_col_by_input_type(InputTypes.ID,
                                                   column_definitions)
target_column = utils.get_single_col_by_input_type(InputTypes.TARGET,
                                                       column_definitions)

#### Encode

In [60]:
# (1) Format real scalers
_real_scalers = {}
for col in ['oil', 'transactions', 'log_sales']:
    _real_scalers[col] = (df[col].mean(), df[col].std())

_target_scaler = (df[target_column].mean(), df[target_column].std())

In [63]:
# (2) Format categorical scalers
categorical_inputs_name = utils.extract_cols_from_data_type(
          DataTypes.CATEGORICAL, real_inputs + categorical_inputs,
          {InputTypes.ID, InputTypes.TIME})

categorical_scalers = {}
num_classes = []

valid_idx = df['traj_id'].apply(lambda x: x in set(identifiers))
for col in categorical_inputs_name:
# Set all to str so that we don't have mixed integer/string columns
    srs = df[col].apply(str).loc[valid_idx]
    categorical_scalers[col] = sklearn.preprocessing.LabelEncoder().fit(
            srs.values)
    num_classes.append(srs.nunique())
num_classes_per_cat_input = num_classes
_cat_scalers = categorical_scalers

In [64]:
print(_real_scalers)
print(_target_scaler)
print(_cat_scalers)

{'oil': (64.74225129198966, 44.23409343597884), 'transactions': (1400.2722868217054, 613.487068343699), 'log_sales': (0.9648701125069206, 0.7171221902090524)}
(0.9648701125069206, 0.7171221902090524)
{'item_nbr': LabelEncoder(), 'store_nbr': LabelEncoder(), 'city': LabelEncoder(), 'state': LabelEncoder(), 'type': LabelEncoder(), 'cluster': LabelEncoder(), 'family': LabelEncoder(), 'class': LabelEncoder(), 'perishable': LabelEncoder(), 'onpromotion': LabelEncoder(), 'day_of_week': LabelEncoder(), 'national_hol': LabelEncoder(), 'regional_hol': LabelEncoder(), 'local_hol': LabelEncoder()}


In [65]:
def transform_inputs(df):
    """Performs feature transformations.

    This includes both feature engineering, preprocessing and normalisation.

    Args:
      df: Data frame to transform.

    Returns:
      Transformed data frame.

    """
    output = df.copy()

    if _real_scalers is None and _cat_scalers is None:
      raise ValueError('Scalers have not been set!')

    #column_definitions = get_column_definition()

    categorical_inputs = utils.extract_cols_from_data_type(
        DataTypes.CATEGORICAL, column_definitions,
        {InputTypes.ID, InputTypes.TIME})

    # (1) Format real inputs: standardization
    for col in ['log_sales', 'oil', 'transactions']:
      mean, std = _real_scalers[col]
      output[col] = (df[col] - mean) / std  # standardization

      if col == 'log_sales':
        output[col] = output[col].fillna(0.)  # mean imputation

    # (2) Format categorical inputs: LabelEncoder()
    for col in categorical_inputs:
      string_df = df[col].apply(str)
      output[col] = _cat_scalers[col].transform(string_df)
    ## Example:
    ## _cat_scalers['city'].transform(train['city'].apply(str))

    return output

In [66]:
train_tf = transform_inputs(train) 
valid_tf = transform_inputs(valid) 
test_tf = transform_inputs(test) 

In [80]:
print(train_tf.shape)
print(valid_tf.shape)
print(test_tf.shape)

(921, 24)
(360, 24)
(360, 24)


#### Export dataset

In [79]:
# export file
train_tf.to_csv('train_tf.csv',index=False)
valid_tf.to_csv('valid_tf.csv',index=False)
test_tf.to_csv('test_tf.csv',index=False)

### Hyperparam Optimization

#### Fixed param

In [35]:
def _get_tft_input_indices():
    """Returns the relevant indexes and input sizes required by TFT."""

    # Functions
    def _get_locations(input_types, defn):
      return [i for i, tup in enumerate(defn) if tup[2] in input_types]

    locations = {
        'input_size': # not a str
            len(real_inputs + categorical_inputs) ,# remove ID, TIME

        'output_size': # loc within total inputs
            len(_get_locations({InputTypes.TARGET}, real_inputs + categorical_inputs)),

        'category_counts':
            num_classes_per_cat_input,
            
        'input_obs_loc': # loc within total inputs
            _get_locations({InputTypes.TARGET}, real_inputs + categorical_inputs),

        'static_input_loc':# loc within total inputs
            _get_locations({InputTypes.STATIC_INPUT}, real_inputs + categorical_inputs),

        'known_regular_inputs':# loc within real_inputs
            _get_locations({InputTypes.STATIC_INPUT, InputTypes.KNOWN_INPUT},
                           real_inputs), 

        'known_categorical_inputs':# loc within categorical_inputs
            _get_locations({InputTypes.STATIC_INPUT, InputTypes.KNOWN_INPUT},
                           categorical_inputs)  
    }
    return locations
fixed_params.update(_get_tft_input_indices())
fixed_params

{'total_time_steps': 120,
 'num_encoder_steps': 90,
 'num_epochs': 100,
 'early_stopping_patience': 5,
 'multiprocessing_workers': 5,
 'column_definition': [('traj_id',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.ID: 4>),
  ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>),
  ('log_sales', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>),
  ('transactions', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>),
  ('oil', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>),
  ('day_of_month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('open', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('item_nbr', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>),
  ('store_nbr', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>),
  ('city', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>),
  ('state', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 

In [22]:
def filter_ids(frame): # must have same traj_id among train, valid and test dataset
    index = frame['traj_id']
    return frame.loc[index.apply(lambda x: x in set(identifiers))] 

In [139]:
# Parameter overrides for testing only! Small sizes used to speed up script.
fixed_params["num_epochs"] = 1
params["hidden_layer_size"] = 5

model_folder = '0615_result'
# model_folder: Folder to store optimisation artifacts.
params["model_folder"] = model_folder

train_samples, valid_samples = 100, 10

# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
num_repeats = 1
use_gpu = False # use cpu

*** Loading hyperparm manager ***


#### Param update

In [140]:
# hyperparam opt
param_ranges = {k: [params[k]] for k in params} # k: name of param, params[k]: value of param
param_ranges

{'dropout_rate': [0.1],
 'hidden_layer_size': [5],
 'learning_rate': [0.001],
 'minibatch_size': [128],
 'max_gradient_norm': [100.0],
 'num_heads': [4],
 'stack_size': [1],
 'model_folder': ['0615_result']}

In [141]:
def get_next_parameters(ranges_to_skip=None):
    """Returns the next set of parameters to optimise.
    Args:
      ranges_to_skip: Explicitly defines a set of keys to skip.
    """
    _max_tries = 1000
    results = pd.DataFrame()

    if ranges_to_skip is None:
      ranges_to_skip = set(results.index)

    param_range_keys = list(param_ranges.keys())
    param_range_keys.sort()

    def _get_next():
      """Returns next hyperparameter set per try."""
      parameters = {
          k: np.random.choice(param_ranges[k]) for k in param_range_keys
      } 
      # Adds fixed params
      for k in fixed_params:
        parameters[k] = fixed_params[k]
      return parameters
    
    def _get_name(params):
      """Returns a unique key for the supplied set of params."""

      #self._check_params(params)

      fields = list(params.keys())
      fields.sort()
      return "_".join([str(params[k]) for k in fields])

    for _ in range(_max_tries):

      parameters = _get_next()
      name = _get_name(parameters)

      if name not in ranges_to_skip:
        return parameters

    raise ValueError("Exceeded max number of hyperparameter searches!!")

params_update = get_next_parameters()

In [142]:
params_update

{'dropout_rate': 0.1,
 'hidden_layer_size': 5,
 'learning_rate': 0.001,
 'max_gradient_norm': 100.0,
 'minibatch_size': 128,
 'model_folder': '0615_result',
 'num_heads': 4,
 'stack_size': 1,
 'total_time_steps': 120,
 'num_encoder_steps': 90,
 'num_epochs': 1,
 'early_stopping_patience': 5,
 'multiprocessing_workers': 5,
 'column_definition': [('traj_id',
   <DataTypes.REAL_VALUED: 0>,
   <InputTypes.ID: 4>),
  ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>),
  ('log_sales', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>),
  ('transactions', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>),
  ('oil', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>),
  ('day_of_month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('open', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
  ('item_nbr', <DataTypes.CATEGORICAL: 1>, <InputTypes.STATIC_INPUT: 3>),
  ('store_nbr', <Data

#### Export params

In [143]:
import pickle 
with open('params_update.pkl', 'wb') as f:
    pickle.dump(params_update, f)

## Other

In [69]:
def format_predictions(self, predictions):
    """Reverts any normalisation to give predictions in original scale.

    Args:
      predictions: Dataframe of model predictions.

    Returns:
      Data frame of unnormalised predictions.
    """
    output = predictions.copy()

    column_names = predictions.columns
    mean, std = self._target_scaler
    for col in column_names:
      if col not in {'forecast_time', 'identifier'}:
        output[col] = (predictions[col] * std) + mean

    return output

In [70]:
#HyperparamOptManager = libs.hyperparam_opt.HyperparamOptManager
ModelClass = libs.tft_model.TemporalFusionTransformer

In [73]:
#%pip install tensorflow==2.5.0
#%pip install keras==2.2.4 

In [74]:
# Sets up hyperparam manager
print("*** Loading hyperparm manager ***")
#opt_manager = HyperparamOptManager({k: [params[k]] for k in params},
#                                     fixed_params, model_folder)

  # Training -- one iteration only
print("*** Running calibration ***")
print("Params Selected:")
for k in params:
    print("{}: {}".format(k, params[k]))

best_loss = np.Inf
for _ in range(num_repeats):

    tf.reset_default_graph()
    with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:

      #tf.keras.backend.set_session(sess)
      K.set_session(sess)

      #params = opt_manager.get_next_parameters()
      params_update = get_next_parameters() # add fixed params
      model = ModelClass(params_update, use_cudnn=use_gpu)

      if not model.training_data_cached():
        model.cache_batched_data(train, "train", num_samples=train_samples)
        model.cache_batched_data(valid, "valid", num_samples=valid_samples)

      sess.run(tf.global_variables_initializer())
      model.fit()

      val_loss = model.evaluate()

      if val_loss < best_loss:
        opt_manager.update_score(params, val_loss, model)
        best_loss = val_loss

      tf.keras.backend.set_session(default_keras_session)

*** Loading hyperparm manager ***
*** Running calibration ***
Params Selected:
dropout_rate: 0.1
hidden_layer_size: 5
learning_rate: 0.001
minibatch_size: 128
max_gradient_norm: 100.0
num_heads: 4
stack_size: 1
model_folder: 0615_result
Resetting temp folder...
*** TemporalFusionTransformer params ***
# dropout_rate = 0.1
# hidden_layer_size = 5
# learning_rate = 0.001
# max_gradient_norm = 100.0
# minibatch_size = 128
# model_folder = 0615_result
# num_heads = 4
# stack_size = 1
# total_time_steps = 120
# num_encoder_steps = 90
# num_epochs = 1
# early_stopping_patience = 5
# multiprocessing_workers = 5
# column_definition = [('traj_id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>), ('date', <DataTypes.DATE: 2>, <InputTypes.TIME: 5>), ('log_sales', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>), ('transactions', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('oil', <DataTypes.REAL_VALUED: 0>, <InputTypes.OBSERVED_INPUT: 1>), ('day_of_month', <DataTypes.REAL_VALU

c:\Users\pan\miniconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
print("*** Running tests ***")
tf.reset_default_graph()
with tf.Graph().as_default(), tf.Session(config=tf_config) as sess:
    tf.keras.backend.set_session(sess)
    best_params = opt_manager.get_best_params()
    model = ModelClass(best_params, use_cudnn=use_gpu)

    model.load(opt_manager.hyperparam_folder)

    print("Computing best validation loss")
    val_loss = model.evaluate(valid)

    print("Computing test loss")
    output_map = model.predict(test, return_targets=True)
    targets = format_predictions(output_map["targets"])
    p50_forecast = format_predictions(output_map["p50"])
    p90_forecast = format_predictions(output_map["p90"])

    def extract_numerical_data(data):
      """Strips out forecast time and identifier columns."""
      return data[[
          col for col in data.columns
          if col not in {"forecast_time", "identifier"}
      ]]

    p50_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p50_forecast),
        0.5)
    p90_loss = utils.numpy_normalised_quantile_loss(
        extract_numerical_data(targets), extract_numerical_data(p90_forecast),
        0.9)

    tf.keras.backend.set_session(default_keras_session)
    

In [ ]:
print("Training completed @ {}".format(dte.datetime.now()))
print("Best validation loss = {}".format(val_loss))
print("Params:")

for k in best_params:
    print(k, " = ", best_params[k])
print()
print("Normalised Quantile Loss for Test Data: P50={}, P90={}".format(
      p50_loss.mean(), p90_loss.mean()))